In [5]:
#Required packages
import time
import re
import pandas as pd
import pymysql
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

# Scroll to the bottom to load all content
def scrolldown():
    previous_page_height=driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') 
        time.sleep(1)
        new_page_height = driver.execute_script("return document.body.scrollHeight") 
        if new_page_height == previous_page_height:
            break
        previous_page_height = new_page_height

#CLICK VIEW_BUSES(2&1) NEW
def click_view_buses():
    try:
        #fetching the container
        container=wait.until(EC.visibility_of_all_elements_located((By.XPATH,"//div[contains(@class, 'button') and contains(text(), 'View Buses')]")))
        #checking the number of view buses and clicking
        for i in range(len(container))[::-1]:
            container[i].click()
            time.sleep(2)
        time.sleep(2)    
    except:
        pass  

#GETTING ROUTE NAMES AND ROUTE_LINKS
def Get_route_deatils(link):
    
    driver.get(link)
    time.sleep(2)
    Route_and_link=[]
    try:
        wait = WebDriverWait(driver, 20)
        pc = wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="DC_117_paginationTable"]')))
        num_pgs =len(driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs "))+1
        if num_pgs==1:
            #getting bus routes names
            Bus_routes=driver.find_elements(By.XPATH,"//a[@class='route']")
            allroute=[i.text for i in Bus_routes]
            
            #getting bus_routes_links
            nested_div=wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.D117_main.D117_container')))
            route_links = nested_div.find_elements(By.CSS_SELECTOR, 'div.route_link')
            link=[i.find_element(By.TAG_NAME,"a").get_attribute('href') for i in route_links]
            
            r=zip(allroute,link)
            Route_and_link.extend(r)
        else:
            for i in range(1,num_pgs):
                #getting bus routes names
                Bus_routes=driver.find_elements(By.XPATH,"//a[@class='route']")
                allroute=[i.text for i in Bus_routes]
                
                #getting bus_routes_links
                nested_div=wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.D117_main.D117_container')))
                route_links = nested_div.find_elements(By.CSS_SELECTOR, 'div.route_link')
                link=[i.find_element(By.TAG_NAME,"a").get_attribute('href') for i in route_links]
                
                r=zip(allroute,link)
                Route_and_link.extend(r)
                
                if i==num_pgs-1:
                    break
                else:
                    next_page= pc.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{i+ 1}"]')
                    action = ActionChains(driver)
                    action.move_to_element(next_page).perform()
                    time.sleep(2)
                    next_page.click()
                    time.sleep(3)
    except:
        pass             
    return Route_and_link    
            
 #Get Bus data for each link by passing here
def Get_bus_details(link):
    
    driver.get(link)
    time.sleep(3)
    route_name=driver.find_element(By.CLASS_NAME,'D136_h1').text
    time.sleep(2)
    click_view_buses()
    time.sleep(2)
    scrolldown()
    wait = WebDriverWait(driver, 20)
    Bus_data=[]
    try:
        bus_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "clearfix.bus-item-details")))
        time.sleep(3)

        for bus in bus_items:
        # Initialize None incase of empty value
            bus_names = bus_type = Departing_time = duration = reaching_time = star_rating = Price = seat_availability = None
        #Bus names
            try:
                bus_names=bus.find_element(By.XPATH,".//div[@class='travels lh-24 f-bold d-color']").text
                
            except:
                pass
        #Bus type
            try:
                bus_type=bus.find_element(By.XPATH,".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                
            except:
                pass
        #Departure time
            try:
                Departing_time=bus.find_element(By.XPATH,".//div[@class='dp-time f-19 d-color f-bold']").text
                
            except:
                pass
        #Duration time
            try:
                duration=bus.find_element(By.XPATH,".//div[@class='dur l-color lh-24']").text
                
            except:
                pass
        #Reaching time
            try:
                reaching_time=bus.find_element(By.XPATH,".//div[@class='bp-time f-19 d-color disp-Inline']").text
                
            except:
                pass
        #star rating
            try:
                rating=bus.find_element(By.XPATH,".//div[@class='rating-sec lh-24']").text
                star_rating=float(rating)
                
            except:
                pass
        # Price
            try:
                price=bus.find_element(By.XPATH,".//div[@class='fare d-block']").text
                price_text=re.findall(r'\d+',price) #using REGULAR_EXPRESSION:Extracting number(string_data_type) from string
                Price= int(price_text[0]) #converting STRING data type to INT data type eg:[["66"]]
            except:
                pass
        # Seat Availability
            try:
                seat_availability=bus.find_element(By.XPATH,".//div[@class='seat-left m-top-30']").text
                seat_available=bus.find_element(By.XPATH,".//div[@class='seat-left m-top-30']").text
                seats_text=re.findall(r'\d+',seat_available)
                seat_availability=int(seats_text[0])
                
            except:
                pass

            Bus_data.append({
                    "Bus_route_link":link,
                    "Bus_name":bus_names,
                    "Bus_type":bus_type,
                    "Departure":Departing_time,
                    "Duration":duration,
                    "Reaching_time":reaching_time,
                    "Rating":star_rating,
                    "Price":Price,
                    "Seats_available":seat_availability })
    except:
        print('Oops! No buses found in:',route_name)
    return Bus_data   

def add_column(x):
    if 'TSRTC' in x:
        return "Government bus"
    elif 'APSRTC' in x:
        return 'Government bus'
    else:
        return 'private bus'

def route_data_to_df(route_data):
    route_data_dict=[]
    for i in range(len(route_data)):
        route_data_dict.append({
            'Route_name':route_data[i][0],
            'Route_link':route_data[i][1]
        })
    route_data_df=pd.DataFrame(route_data_dict)
    return route_data_df  

def fetch_all_state_data(state_link):
    
    route_data=Get_route_deatils(state_link) # Getting route details
    route_links=[i[1] for i in route_data] # Getting route links
    route_data_df=route_data_to_df(route_data) # getting route_data as DataFrame
    bus_data=[] # Going to append bus data [dict]
    for link in route_links:                                                          [{}]
        list_dict=Get_bus_details(link) # Getting bus details
        if len(list_dict)!=0: #checking if there was an empty list incase of page unavailable
            for dict in list_dict:
                bus_data.append(dict)
    bus_data_df=pd.DataFrame(bus_data)  
    bus_data_df['Bus_category']=bus_data_df['Bus_name'].apply(add_column)
     
    return bus_data_df,route_data_df

#pushing data to MYSQL
def pushing_to_sql(df):
    df=df.fillna(0)    
    mycursor.execute("create database if not exists Redbus_project")
    mycursor.execute("use Redbus_project")
    mycursor.execute("""create table if not exists BUS_TABLE(
                        Bus_route_link TEXT,
                        Bus_name TEXT,
                        Bus_type TEXT,
                        Departure TEXT,
                        Duration TEXT,
                        Reaching_time TIME,
                        Rating FLOAT,
                        Price INT,
                        Seats_available FLOAT,
                        Bus_category TEXT,
                        Route_name TEXT,
                        State TEXT ) """)
    
    for index,row in df.iterrows():
        sql="insert into BUS_TABLE(Bus_route_link,Bus_name,Bus_type,Departure,Duration,Reaching_time,Rating,Price,Seats_available,Bus_category,Route_name,State)values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        val=(row['Bus_route_link'],row['Bus_name'],row['Bus_type'],row['Departure'],row['Duration'],row['Reaching_time'],
            row['Rating'],row['Price'],row['Seats_available'],row['Bus_category'],row['Route_name'],row['State'])
        mycursor.execute(sql,val)
    myconnection.commit()

#MAIN CODE
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

#WBSTC-1pg(West bengal)
WBSTC_buses_df,WBSTC_route_df = fetch_district_data('https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtch')
WBSTC_df = WBSTC_buses_df.merge(WBSTC_route_df,left_on='Bus_route_link',right_on='Route_link',how='inner')

#JKSRTC-2pg(Jammu and Kashmir)
JKSRTC_buses_df,JKSRTC_route_df = fetch_district_data('https://www.redbus.in/online-booking/jksrtc')
JKSRTC_df = JKSRTC_buses_df.merge(JKSRTC_route_df,left_on='Bus_route_link',right_on='Route_link',how='inner')

#KAAC-2pg(Karbi Anglong)
KAAC_buses_df,KAAC_route_df = fetch_district_data('https://www.redbus.in/online-booking/kaac-transport')
KAAC_df = KAAC_buses_df.merge(KAAC_route_df,left_on='Bus_route_link',right_on='Route_link',how='inner')

#KERALA RTC-2pg(kerala)
KERALA_buses_df,KERALA_route_df = fetch_district_data('https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile')
KERALA_df = KERALA_buses_df.merge(KERALA_route_df,left_on='Bus_route_link',right_on='Route_link',how='inner')

#PEPSU-3pg(Punjab)
PEPSU_buses_df,PEPSU_route_df = fetch_district_data('https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile')
PEPSU_df = PEPSU_buses_df.merge(PEPSU_route_df,left_on='Bus_route_link',right_on='Route_link',how='inner')

#RSRTC-3pg(Rajasthan)
RSRTC_buses_df,RSRTC_route_df = fetch_district_data('https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile')
RSRTC_df = RSRTC_buses_df.merge(RSRTC_route_df,left_on='Bus_route_link',right_on='Route_link',how='inner')

#TSRTC-3pg(Telangana)
TSRTC_buses_df,TSRTC_route_df = fetch_district_data('https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile')
TSRTC_df = TSRTC_buses_df.merge(TSRTC_route_df,left_on='Bus_route_link',right_on='Route_link',how='inner')

#KTCL-4pg(Kadamba)
KTCL_buses_df,KTCL_route_df = fetch_district_data('https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile')
KTCL_df = KTCL_buses_df.merge(KTCL_route_df,left_on='Bus_route_link',right_on='Route_link',how='inner')

#BSRTC-4pg(Bihar)
BSRTC_buses_df,BSRTC_route_df = fetch_district_data('https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile')
BSRTC_df = BSRTC_buses_df.merge(BSRTC_route_df,left_on='Bus_route_link',right_on='Route_link',how='inner')

#APSRTC-5pg
APSRTC_buses_df,APSRTC_route_df = fetch_all_state_data('https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile')
APSRTC_df = APSRTC_buses_df.merge(APSRTC_route_df,left_on='Bus_route_link',right_on='Route_link',how='inner')



#ADDING STATE COLUMN FOR EVERY DFs
WBSTC_df['State']='West bengal'
JKSRTC_df['State']='Jammu and Kashmir'
KAAC_df['State']='Karbi Anglong'
KERALA_df['State']='kerala'
PEPSU_df['State']='Punjab'
RSRTC_df['State']='Rajasthan'
TSRTC_df['State']='Telangana'
KTCL_df['State']='Kadamba'
BSRTC_df['State']='Bihar'
APSRTC_df['State']='Andhra Pradesh'

#Connecting VS-CODE to mysql
myconnection=pymysql.connect(host='localhost',user='root',passwd='root',port=3306)
mycursor=myconnection.cursor()

#PUSHING EVERY DF TO MYSQL
pushing_to_sql(WBSTC_df)
pushing_to_sql(JKSRTC_df)
pushing_to_sql(KAAC_df)
pushing_to_sql(KERALA_df)
pushing_to_sql(PEPSU_df)
pushing_to_sql(RSRTC_df)
pushing_to_sql(TSRTC_df)
pushing_to_sql(KTCL_df)
pushing_to_sql(BSRTC_df)
pushing_to_sql(APSRTC_df)

mycursor.close()

#NEXT STREAMLIT PART